In [7]:
import numpy as np

import scipy as sp
import scipy.sparse
import scipy.sparse.linalg

import sklearn
import sklearn.preprocessing

import pickle
import random
import newton
import webbrowser

In [8]:
# twitter dataset

with open("ovchinnikov-rutwitterdataset/A.pkl", "rb") as file:
    A = np.load(file, encoding="latin1")

with open("ovchinnikov-rutwitterdataset/labeled_nodes.pkl", "rb") as file:
    labels = np.load(file, encoding="latin1")

with open("ovchinnikov-rutwitterdataset/i2t.pkl", "rb") as file:
    i2t = pickle.load(file)

with open("ovchinnikov-rutwitterdataset/t2i.pkl", "rb") as file:
    t2i = pickle.load(file)

In [9]:
# preprocessing

F = sklearn.preprocessing.normalize(A, axis=1, norm='l1')
B = sklearn.preprocessing.normalize(A.T, axis=1, norm='l1')
d = np.array(labels.todense())[:, 0]

Fcsc = F.tocsc()
Bcsc = B.tocsc()

C:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:420: DataConversionWarning: Data with input dtype int8 was converted to float64 by the normalize function.
  warnings.warn(msg, DataConversionWarning)
C:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:420: DataConversionWarning: Data with input dtype int8 was converted to float64 by the normalize function.
  warnings.warn(msg, DataConversionWarning)


In [10]:
# data for cross-validation

n_n_elements = d[d < 0].shape[0]
n_p_elements = d[d > 0].shape[0]

print("Spam nodes:\t{}".format(n_n_elements))
print("Not spam nodes:\t{}".format(n_p_elements))

n_indices = random.sample(np.where(d < 0)[0].tolist(), int(n_n_elements * .20 // 1))  # random indices of 20% negative
p_indices = random.sample(np.where(d > 0)[0].tolist(), int(n_p_elements * .20 // 1))  # random indices of 20% positive

d_CV = d.copy()
d_CV[n_indices] = 0
d_CV[p_indices] = 0

print("Spam nodes (80%):\t{}".format(d_CV[d_CV < 0].shape[0]))
print("Not spam nodes (80%):\t{}".format(d_CV[d_CV > 0].shape[0]))

def CV(x):
    return float(sum(x[p_indices] > 0) + sum(x[n_indices] < 0)) / (len(p_indices) + len(n_indices))

Spam nodes:	2749
Not spam nodes:	375
Spam nodes (80%):	2200
Not spam nodes (80%):	300


In [14]:
file = open("callback.txt", "w")
file.write("Iter\tErr\tCV\tDer\n")

def cbk(k, x):
    file.write("{}\t{}\t{}\t{}\n".format(str(k), str(np.linalg.norm(x - newton.T(F, B, d_CV, x))), str(CV(x)), str(np.linalg.norm(newton.der(F, B, d_CV, x)))))

In [15]:
k, ans = newton.Newton(F, B, d_CV, tol=1e-1, callback=cbk)

In [16]:
file.close()

In [18]:
# RepRank via Newton answer

k, ans = newton.Newton(F, B, d, tol=1e-1)
print("Required numeber of iterations for original RepRank: {}.".format(str(k)))

for i in list(map(i2t.get, ans.argsort(axis=0)[::-1][:3].tolist())):
    webbrowser.open_new_tab("https://twitter.com/intent/user?user_id={}".format(str(i)))

for i in list(map(i2t.get, ans.argsort(axis=0)[::-1][-3:].tolist())):
    webbrowser.open_new_tab("https://twitter.com/intent/user?user_id={}".format(str(i)))

Required numeber of iterations for original RepRank: 9.


In [19]:
# original RepRank answer

k, ans = newton.RepRank(F, B, d, tol=1e-1)
print("Required numeber of iterations for original RepRank: {}.".format(str(k)))

for i in list(map(i2t.get, ans.argsort(axis=0)[::-1][:3].tolist())):
    webbrowser.open_new_tab("https://twitter.com/intent/user?user_id={}".format(str(i)))

for i in list(map(i2t.get, ans.argsort(axis=0)[::-1][-3:].tolist())):
    webbrowser.open_new_tab("https://twitter.com/intent/user?user_id={}".format(str(i)))

Required numeber of iterations for original RepRank: 14.
